In [2]:
import codecs
import json
import re
import yaml
import os
from tqdm.notebook import tqdm
from PIL import Image
from base64 import b64decode
from io import BytesIO
import numpy as np

同じ文字が連続する場合は長い方から処理する．
- ToDo: citationをどうにかする．

In [3]:
s = '# aa\n ## aaa\n `this` is ``` **sample string** for *extracting substring*. {cite:p}`Echeveste2020-sh` <a>'

In [4]:
def markdown2latex(s):
    # s: string
    s = re.sub(r'\###\ (.+?)\n', r'\\subsubsection{\1}', s)  # subsubsection
    s = re.sub(r'\##\ (.+?)\n', r'\\subsection{\1}', s)  # subsection
    s = re.sub(r'\#\ (.+?)\n', r'\\section{\1}', s)      # section
    
    s = re.sub(r'\*\*(.+?)\*\*', r'\\textbf{\1}', s)   # bold
    #s = re.sub(r'\*(.+?)\*', r'\\textit{\1}', s)       # italic
    
    s = s.replace(r"```{note}", "\\footnote{") # note to footnote
    s = s.replace(r"```", "}")
    s = re.sub(r'<(.+?)>', r'\\url{\1}', s) # url

    s = re.sub(r'{cite:p}`(.+?)`', r'\\cite{\1}', s)     
    s = re.sub(r'`(.+?)`', r'\\jl{\1}', s) # inline code with \newcommand{\jl}{\lstinline[language=julia]}

    s = re.sub(r'<(.+?)>', r'\\url{\1}', s) # url
    s = s.replace(r"（", " (") 
    s = s.replace(r"）", ") ") 
    return s

In [5]:
markdown2latex(s)

'\\section{aa} \\subsection{aaa} \\jl{this} is } \\textbf{sample string} for *extracting substring*. \\cite{Echeveste2020-sh} \\url{a}'

変換

In [6]:
dir_path = "../contents/"

In [7]:
with open(dir_path + "_toc.yml") as file:
    toc_yaml = yaml.safe_load(file)

In [8]:
toc_yaml['sections']

[{'file': 'preface'},
 {'file': 'introduction/intro',
  'sections': [{'file': 'introduction/computational-neuroscience'},
   {'file': 'introduction/notation'},
   {'file': 'introduction/usage-julia-lang'},
   {'file': 'introduction/linear-algebra'},
   {'file': 'introduction/differential-equation'},
   {'file': 'introduction/linear-regression'},
   {'file': 'introduction/probability-information-theory'},
   {'file': 'introduction/stochastic-process-differential-equation'}]},
 {'file': 'neuron-model/intro',
  'sections': [{'file': 'neuron-model/neuron-physiol'},
   {'file': 'neuron-model/hodgkin-huxley'},
   {'file': 'neuron-model/fhn'},
   {'file': 'neuron-model/lif'},
   {'file': 'neuron-model/izhikevich'},
   {'file': 'neuron-model/isi'},
   {'file': 'neuron-model/neurite-growth-model'}]},
 {'file': 'synapse-model/intro',
  'sections': [{'file': 'synapse-model/synapse-physiol'},
   {'file': 'synapse-model/current-conductance-synapse'},
   {'file': 'synapse-model/expo-synapse'},
   {'

In [10]:
def all_remove(xlist, remove):
        return [value for value in xlist if value != remove]

In [11]:
def md_ipynb2latex(dir_path, filename):
    save_dir = "./text/" + filename
    os.makedirs(save_dir, exist_ok=True)
    file_path = dir_path + filename
    master_list = []
    if os.path.isfile(file_path + ".md"):
        f = codecs.open(file_path + ".md", 'r', encoding="utf8")
        md = f.read()
        # convert
        text = markdown2latex(md)
        text = all_remove(text, "\n")
        if not ':filter: docname in docnames\n' in text:
            # save
            parted_file_path = save_dir + "/000.tex"
            with open(parted_file_path, 'w', encoding='UTF-8') as f:
                f.writelines(text)
                master_list.append(r"\input{"+parted_file_path+"}\n")
    elif os.path.isfile(file_path + ".ipynb"):
        f = codecs.open(file_path + ".ipynb", 'r', encoding="utf8")
        source = f.read()
        y = json.loads(source)
        num_cells = len(y['cells'])
        for cell_idx in range(num_cells):
            cell = y['cells'][cell_idx]
            if cell['cell_type'] == 'markdown':
                # convert
                text = [markdown2latex(s) for s in cell['source']]
                text = all_remove(text, "\n")
                if not ':filter: docname in docnames\n' in text:
                    # save
                    parted_file_path = save_dir + "/{:03d}.tex".format(cell_idx)
                    with open(parted_file_path, 'w', encoding='UTF-8') as f:
                        f.writelines(text)
                    master_list.append(r"\input{"+parted_file_path+"}\n")
            elif cell['cell_type'] == 'code':
                # ToDo:'outputs'
                code = cell['source']
                parted_file_path = save_dir + "/{:03d}.jl".format(cell_idx)
                with open(parted_file_path, 'w', encoding='UTF-8') as f:
                    f.writelines(code)
                master_list.append(r"\lstinputlisting[language=julia]{"+parted_file_path+"}\n")

                if cell['outputs']:
                    if 'data' in cell['outputs'][0]:
                        output = cell['outputs'][0]['data']
                        if "image/png" in output.keys():
                            png_bytes = output['image/png']
                            png_bytes = b64decode(png_bytes)
                            bytes_io = BytesIO(png_bytes)
                            image = Image.open(bytes_io)

                            figname = "cell{:03d}.png".format(cell_idx)
                            figsavepath = "./fig/" + filename + "/" + figname
                            os.makedirs("./fig/" + filename, exist_ok=True)
                            image.save(figsavepath, 'png')

                            caption = figname
                            figlabel = figname #"ccc"
                            figcode = "\\begin{figure}[ht]\n\t\centering\n"
                            figcode += "\t\includegraphics[scale=0.8, max width=\linewidth]{"+figsavepath+"}\n"
                            figcode += "\t\caption{" + caption + "}\n"
                            figcode += "\t\label{"+figlabel+"}\n"
                            figcode += "\end{figure}"

                            parted_output_path = save_dir + "/output_{:03d}.tex".format(cell_idx)
                            with open(parted_output_path, 'w', encoding='UTF-8') as f:
                                f.writelines(figcode)
                            master_list.append(r"\input{"+parted_output_path+"}\n")
                        
                        elif "text/plain" in output.keys():
                            print(output["text/plain"])

    with open(save_dir + '.tex', 'w', encoding='UTF-8') as f:
        f.writelines(master_list)

In [54]:
def latex_itemized(text):
    #splited_text = text.split('\n')
    #splited_text = all_remove(splited_text, "\n")
    splited_text = list(filter(None, text))
    item_idx = [line[0] == "-" for line in splited_text]
    if np.sum(item_idx) > 0:
        item_idx += [False]
        item_startend = np.where(np.diff(np.array(item_idx)) == True)[0]
        item_startend += np.arange(len(item_startend)) + 1

        # replace - to \item
        for i in range(len(splited_text)):
            if item_idx[i]:
                splited_text[i] = splited_text[i].replace('-', '\item', 1) 

        # add begin and end
        for j in range(len(item_startend)):
            if j % 2 == 0:
                splited_text.insert(item_startend[j], "\\begin{itemize}")
            else:
                splited_text.insert(item_startend[j], "\\end{itemize}")
    #text2 = '\n'.join(splited_text)
    for i in range(len(splited_text)):
        if splited_text[i][-1:] != "\n":
            splited_text[i] += "\n"
    return splited_text

In [63]:
filename = "introduction/notation"
file_path = dir_path + filename
f = codecs.open(file_path + ".md", 'r', encoding="utf8")
md = f.read()
text = markdown2latex(md)
text = text.split('\n')

In [66]:
text

['\\section{記号の表記}',
 '本書では次のような記号表記を用いる．',
 '- 実数全体を$\\mathbb{R}$, 複素数全体は$\\mathbb{C}$と表記する．',
 '- スカラーは小文字・斜体で $x$ のように表記する．',
 '- ベクトルは小文字・立体・太字で $\\mathbf{x}$ のように表記し，列ベクトル (縦ベクトル) として扱う．',
 '- 行列は大文字・立体・太字で $\\mathbf{X}$ のように表記する．',
 '- $n\\times 1$の実ベクトルの集合を $\\mathbb{R}^n$, $n\\times m$ の実行列の集合を $\\mathbb{R}^{n\\times m}$と表記する．',
 '- 行列 $\\mathbf{X}$ の置換は $\\mathbf{X}^\\top$と表記する．ベクトルの要素を表す場合は $\\mathbf{x} = (x_1, x_2,\\cdots, x_n)^\\top$のように表記する．',
 '- 単位行列を $\\mathbf{I}$ と表記する．',
 '- ゼロベクトルは $\\mathbf{0}$ , 要素が全て1のベクトルは $\\mathbf{1}$ と表記する．  ',
 '- $e$を自然対数の底とし，指数関数を $e^x=\\exp(x)$と表記する．また，自然対数を $\\ln(x)$と表記する．',
 '- 定義を$:=$を用いて行う．例えば，$f(x):=2x$は$f(x)$という関数を$2x$として定義するという意味である．',
 '- 平均 $\\mu$, 標準偏差 $\\sigma$ の正規分布を $\\mathcal{N}(\\mu, \\sigma^2)$ と表記する．',
 '',
 '\\subsection{変数の命名規則}',
 '- \\jl{tp1, tm1} : time plus one (t+1), time minus one (t-1)']

In [71]:
num_cells

26

In [78]:
filename = "energy-based-model/predictive-coding"
file_path = dir_path + filename
f = codecs.open(file_path + ".ipynb", 'r', encoding="utf8")
source = f.read()
y = json.loads(source)
num_cells = len(y['cells'])
cell_idx = 10
cell = y['cells'][cell_idx]
text = [markdown2latex(s) for s in cell['source']]

In [79]:
tex2 = latex_itemized(text)

In [80]:
":filter: docname in docnames" in "".join(tex2)

False

In [58]:
tex2

['\\section{予測符号化}\n',
 '\\subsection{観測世界の階層的予測}\n',
 '\\textbf{階層的予測符号化(hierarchical predictive coding; HPC)} は\\cite{Rao1999-zv}により導入された．構築するネットワークは入力層を含め，3層のネットワークとする．LGNへの入力として画像 $\\mathbf{x} \\in \\mathbb{R}^{n_0}$を考える．画像 $\\mathbf{x}$ の観測世界における隠れ変数，すなわち\\textbf{潜在変数} (latent variable)を$\\mathbf{r} \\in \\mathbb{R}^{n_1}$とし，ニューロン群によって発火率で表現されているとする (真の変数と $\\mathbf{r}$は異なるので文字を分けるべきだが簡単のためにこう表す)．このとき，\n',
 '\n',
 '$$\n',
 '\\mathbf{x} = f(\\mathbf{U}\\mathbf{r}) + \\boldsymbol{\\epsilon} \\tag{1}\n',
 '$$\n',
 '\n',
 'が成立しているとする．ただし，$f(\\cdot)$は活性化関数 (activation function)，$\\mathbf{U} \\in \\mathbb{R}^{n_0 \\times n_1}$は重み行列である．\n',
 '$\\boldsymbol{\\epsilon} \\in \\mathbb{R}^{n_0}$ は $\\mathcal{N}(\\mathbf{0}, \\sigma^2 \\mathbf{I})$ からサンプリングされるとする．\n',
 '\n',
 '潜在変数 $\\mathbf{r}$はさらに高次 (higher-level)の潜在変数 $\\mathbf{r}^h$により，次式で表現される．\n',
 '\n',
 '$$\n',
 '\\mathbf{r} = \\mathbf{r}^{td}+\\boldsymbol{\\epsilon}^{td}=f(\\mathbf{U}^h \\mathbf{r}^h)+\\boldsymbol{\\epsilon}^{td} \\ta

In [51]:
for i in range(len(tex2)):
    if tex2[i][-1:] != "\n":
        tex2[i] += "\n"

In [52]:
tex2

['\\section{予測符号化}\n',
 '\\subsection{観測世界の階層的予測}\n',
 '\\textbf{階層的予測符号化(hierarchical predictive coding; HPC)} は\\cite{Rao1999-zv}により導入された．構築するネットワークは入力層を含め，3層のネットワークとする．LGNへの入力として画像 $\\mathbf{x} \\in \\mathbb{R}^{n_0}$を考える．画像 $\\mathbf{x}$ の観測世界における隠れ変数，すなわち\\textbf{潜在変数} (latent variable)を$\\mathbf{r} \\in \\mathbb{R}^{n_1}$とし，ニューロン群によって発火率で表現されているとする (真の変数と $\\mathbf{r}$は異なるので文字を分けるべきだが簡単のためにこう表す)．このとき，\n',
 '\n',
 '$$\n',
 '\\mathbf{x} = f(\\mathbf{U}\\mathbf{r}) + \\boldsymbol{\\epsilon} \\tag{1}\n',
 '$$\n',
 '\n',
 'が成立しているとする．ただし，$f(\\cdot)$は活性化関数 (activation function)，$\\mathbf{U} \\in \\mathbb{R}^{n_0 \\times n_1}$は重み行列である．\n',
 '$\\boldsymbol{\\epsilon} \\in \\mathbb{R}^{n_0}$ は $\\mathcal{N}(\\mathbf{0}, \\sigma^2 \\mathbf{I})$ からサンプリングされるとする．\n',
 '\n',
 '潜在変数 $\\mathbf{r}$はさらに高次 (higher-level)の潜在変数 $\\mathbf{r}^h$により，次式で表現される．\n',
 '\n',
 '$$\n',
 '\\mathbf{r} = \\mathbf{r}^{td}+\\boldsymbol{\\epsilon}^{td}=f(\\mathbf{U}^h \\mathbf{r}^h)+\\boldsymbol{\\epsilon}^{td} \\ta

In [53]:
with open("test.txt", 'w', encoding='UTF-8') as f:
    f.writelines(tex2)

In [ ]:
latex_itemized

In [18]:
text 

['\\section{予測符号化}',
 '\\subsection{観測世界の階層的予測}',
 '\\textbf{階層的予測符号化(hierarchical predictive coding; HPC)} は\\cite{Rao1999-zv}により導入された．構築するネットワークは入力層を含め，3層のネットワークとする．LGNへの入力として画像 $\\mathbf{x} \\in \\mathbb{R}^{n_0}$を考える．画像 $\\mathbf{x}$ の観測世界における隠れ変数，すなわち\\textbf{潜在変数} (latent variable)を$\\mathbf{r} \\in \\mathbb{R}^{n_1}$とし，ニューロン群によって発火率で表現されているとする (真の変数と $\\mathbf{r}$は異なるので文字を分けるべきだが簡単のためにこう表す)．このとき，',
 '',
 '$$',
 '\\mathbf{x} = f(\\mathbf{U}\\mathbf{r}) + \\boldsymbol{\\epsilon} \\tag{1}',
 '$$',
 '',
 'が成立しているとする．ただし，$f(\\cdot)$は活性化関数 (activation function)，$\\mathbf{U} \\in \\mathbb{R}^{n_0 \\times n_1}$は重み行列である．',
 '$\\boldsymbol{\\epsilon} \\in \\mathbb{R}^{n_0}$ は $\\mathcal{N}(\\mathbf{0}, \\sigma^2 \\mathbf{I})$ からサンプリングされるとする．',
 '',
 '潜在変数 $\\mathbf{r}$はさらに高次 (higher-level)の潜在変数 $\\mathbf{r}^h$により，次式で表現される．',
 '',
 '$$',
 '\\mathbf{r} = \\mathbf{r}^{td}+\\boldsymbol{\\epsilon}^{td}=f(\\mathbf{U}^h \\mathbf{r}^h)+\\boldsymbol{\\epsilon}^{td} \\tag{2}',
 '$$',
 '',
 'ただし，Top

In [100]:


md = f.read()
# convert
text = markdown2latex(md)
#text = all_remove(text, "\n")

In [116]:
text = r"""##Hamiltonianネットワークの方が安定して事後分布を推定することができている．ToDo: 以下の記述

- ここでは重みを設定したが， {cite:p}`Echeveste2020-sh`ではRNNにBPTTで重みを学習させている．
- 動的な入力に対するサンプリング {cite:p}`Berkes2011-xj`．burn-inがなくなり効率良くサンプリングできる．
"""

In [117]:
text

'##Hamiltonianネットワークの方が安定して事後分布を推定することができている．ToDo: 以下の記述\n\n- ここでは重みを設定したが， {cite:p}`Echeveste2020-sh`ではRNNにBPTTで重みを学習させている．\n- 動的な入力に対するサンプリング {cite:p}`Berkes2011-xj`．burn-inがなくなり効率良くサンプリングできる．\n'

In [118]:
text = markdown2latex(text)

In [119]:
latex_itemized(text)

'##Hamiltonianネットワークの方が安定して事後分布を推定することができている．ToDo: 以下の記述\n\\begin{itemize}\n\\item ここでは重みを設定したが， \\cite{Echeveste2020-sh}ではRNNにBPTTで重みを学習させている．\n\\item 動的な入力に対するサンプリング \\cite{Berkes2011-xj}．burn-inがなくなり効率良くサンプリングできる．\n\\end{itemize}'

array([ 2, 14, 16, 18], dtype=int64)

In [91]:
text2

'\\section{記号の表記}\n本書では次のような記号表記を用いる．\n\\begin{itemize}\n\\item 実数全体を$\\mathbb{R}$, 複素数全体は$\\mathbb{C}$と表記する．\n\\item スカラーは小文字・斜体で $x$ のように表記する．\n\\item ベクトルは小文字・立体・太字で $\\mathbf{x}$ のように表記し，列ベクトル (縦ベクトル) として扱う．\n\\item 行列は大文字・立体・太字で $\\mathbf{X}$ のように表記する．\n\\item $n\\times 1$の実ベクトルの集合を $\\mathbb{R}^n$, $n\\times m$ の実行列の集合を $\\mathbb{R}^{n\\times m}$と表記する．\n\\item 行列 $\\mathbf{X}$ の置換は $\\mathbf{X}^\\top$と表記する．ベクトルの要素を表す場合は $\\mathbf{x} = (x_1, x_2,\\cdots, x_n)^\\top$のように表記する．\n\\item 単位行列を $\\mathbf{I}$ と表記する．\n\\item ゼロベクトルは $\\mathbf{0}$ , 要素が全て1のベクトルは $\\mathbf{1}$ と表記する．  \n\\item $e$を自然対数の底とし，指数関数を $e^x=\\exp(x)$と表記する．また，自然対数を $\\ln(x)$と表記する．\n\\item 定義を$:=$を用いて行う．例えば，$f(x):=2x$は$f(x)$という関数を$2x$として定義するという意味である．\n\\item 平均 $\\mu$, 標準偏差 $\\sigma$ の正規分布を $\\mathcal{N}(\\mu, \\sigma^2)$ と表記する．\n\\end{itemize}\n\\subsection{変数の命名規則}\n\\begin{itemize}\n\\item \\jl{tp1, tm1} : time plus one (t+1), time minus one (t-1)\n\\end{itemize}'

In [44]:
line[0]

'-'

In [42]:
item_idx += [False]

In [50]:
splited_text.insert(2, "\\begin{itemize}")

In [51]:
splited_text

['\\section{記号の表記}',
 '本書では次のような記号表記を用いる．',
 '\\begin{itemize}',
 '\x08egin{itemize}',
 '\\item 実数全体を$\\mathbb{R}$, 複素数全体は$\\mathbb{C}$と表記する．',
 '\\item スカラーは小文字・斜体で $x$ のように表記する．',
 '\\item ベクトルは小文字・立体・太字で $\\mathbf{x}$ のように表記し，列ベクトル (縦ベクトル) として扱う．',
 '\\item 行列は大文字・立体・太字で $\\mathbf{X}$ のように表記する．',
 '\\item $n\\times 1$の実ベクトルの集合を $\\mathbb{R}^n$, $n\\times m$ の実行列の集合を $\\mathbb{R}^{n\\times m}$と表記する．',
 '\\item 行列 $\\mathbf{X}$ の置換は $\\mathbf{X}^\\top$と表記する．ベクトルの要素を表す場合は $\\mathbf{x} = (x_1, x_2,\\cdots, x_n)^\\top$のように表記する．',
 '\\item 単位行列を $\\mathbf{I}$ と表記する．',
 '\\item ゼロベクトルは $\\mathbf{0}$ , 要素が全て1のベクトルは $\\mathbf{1}$ と表記する．  ',
 '\\item $e$を自然対数の底とし，指数関数を $e^x=\\exp(x)$と表記する．また，自然対数を $\\ln(x)$と表記する．',
 '\\item 定義を$:=$を用いて行う．例えば，$f(x):=2x$は$f(x)$という関数を$2x$として定義するという意味である．',
 '\\item 平均 $\\mu$, 標準偏差 $\\sigma$ の正規分布を $\\mathcal{N}(\\mu, \\sigma^2)$ と表記する．',
 '\\subsection{変数の命名規則}',
 '\\item \\jl{tp1, tm1} : time plus one (t+1), time minus one (t-1)']

In [34]:
for i, line in enumerate(splited_text):
    if line:
        print(line[0] == "-")

False
False
True
True
True
True
True
True
True
True
True
True
True
False
True


In [13]:
            main_list.append(r"\input{./text/"+filename+".tex}\n")

0it [00:00, ?it/s]

preface
introduction/intro
introduction/computational-neuroscience
introduction/notation
introduction/usage-julia-lang
['2']
['10']
['right! (generic function with 1 method)']
['foo (generic function with 1 method)']
introduction/linear-algebra
['3-element Vector{Int64}:\n', ' 1\n', ' 2\n', ' 3']
['Any[]']
['3×3 Matrix{Float64}:\n', ' 0.926253  0.353171   0.343478\n', ' 0.62924   0.0944723  0.917437\n', ' 0.680284  0.170864   0.654275']
['3×3 Matrix{Float64}:\n', ' 0.414287  0.571006  0.179344\n', ' 0.281441  0.152742  0.479032\n', ' 0.304272  0.276252  0.341624']
['2×2 Matrix{Int64}:\n', ' 1  2\n', ' 3  4']
['2×3 Matrix{Int64}:\n', ' 4  5  6\n', ' 7  8  9']
['2×5 Matrix{Int64}:\n', ' 1  2  4  5  6\n', ' 3  4  7  8  9']
['2×5 Matrix{Int64}:\n', ' 1  2  4  5  6\n', ' 3  4  7  8  9']
['2-element Vector{Matrix{Int64}}:\n', ' [1 2; 3 4]\n', ' [4 5 6; 7 8 9]']
['5×2 Matrix{Int64}:\n', ' 1  2\n', ' 3  4\n', ' 4  7\n', ' 5  8\n', ' 6  9']
['5×2 Matrix{Int64}:\n', ' 1  2\n', ' 3  4\n', ' 4  7\

['normal_equation (generic function with 1 method)']
['kde (generic function with 3 methods)']
appendix/intro
appendix/grid-cells-decoding
['MAT.MAT_v5.Matlabv5File(IOStream(<file ../_static/datasets/grid_cells_data/10704-07070407_T2C3.mat>), false, #undef)']
['nearest_pos (generic function with 1 method)']
appendix/graph-theory-network-model
["PyObject <module 'networkx' from 'C:\\\\Users\\\\yamta\\\\miniconda3\\\\lib\\\\site-packages\\\\networkx\\\\__init__.py'>"]
appendix/useful-links
appendix/usage-jupyter-book
